# Install Dependencies

## External Dependencies

In [1]:
!pip install pandas


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


## FrostAura Dependencies

In [2]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

In [3]:
%run ./asset_valuation_engine.finviz.ipynb

Note: you may need to restart the kernel to use updated packages.


DEBUG:matplotlib:matplotlib data path: /Users/deanmartin/miniforge3/envs/tensorflow-apple-metal/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/Users/deanmartin/.matplotlib
DEBUG:matplotlib:matplotlib version 3.4.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is darwin
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', '_heapq', 'heapq', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections',

In [4]:
%run ./asset_calculations_engine.simple.ipynb


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


DEBUG:root:Calculating profit for asset "TSLA".
INFO:root:Fetching historical price movements for symbol "TSLA".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443


Note: you may need to restart the kernel to use updated packages.


DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/TSLA?period1=-631159200&period2=1660730161&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None
DEBUG:root:[TSLA] Processing transaction value $0.0688 on 2022-07-28 00:00:00.
DEBUG:root:Calculating profit for asset "AAPL".
INFO:root:Fetching historical price movements for symbol "AAPL".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/AAPL?period1=-631159200&period2=1660730162&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None
DEBUG:root:[AAPL] Processing transaction value $0.4317 on 2022-07-01 00:00:00.
DEBUG:root:Calculating profit for asset "DDD".
INFO:root:Fetching historical price movements for symbol "DDD".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
DEBUG:urllib3.connection

{'percentage': 6.484225703326896, 'value': 16.745661399269096}


In [5]:
from frostaura.engines import IAssetProjectionEngine
from frostaura.engines import SimpleAssetProjectionEngine
import pandas as pd
from frostaura.engines import IAssetValuationEngine
from frostaura.engines import FinvizAssetValuationEngine
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import HtmlResourcesDataAccess
from frostaura.models import ValuationResult

# Usage

In [6]:
instance: IAssetProjectionEngine = SimpleAssetProjectionEngine()
html_data_access: IResourcesDataAccess = HtmlResourcesDataAccess()
asset_valuation_engine: IAssetValuationEngine = FinvizAssetValuationEngine(html_data_access=html_data_access)

In [7]:
n_months: int = 12*30 # 39 year
usd_zar_exchange_rate: float = 16.17
annual_growth_rate: float = valuation.eps_five_years + (valuation.annual_dividend_percentage / 100)
initial_value: float = holdings_profits.loc[holdings_profits['symbol'] == 'AAPL']['total_current_usd'].sum()
growth: pd.DataFrame = instance.project_monthly_asset_growth(n_months=n_months,
                                                             annual_growth_rate=annual_growth_rate,
                                                             principal_value=initial_value,
                                                             monthly_deposit=(1000 / usd_zar_exchange_rate / 4))

growth

,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
month,,,,,
0,0.00000,0.000000,0.000000,0.000000,74.697050
1,15.46073,0.644885,15.460730,0.644885,90.802665
2,15.46073,0.783930,30.921459,1.428814,107.047324
3,15.46073,0.924175,46.382189,2.352989,123.432229
4,15.46073,1.065632,61.842919,3.418621,139.958590
...,...,...,...,...,...
356,15.46073,325.186372,5504.019790,32428.305405,38007.022245
357,15.46073,328.127292,5519.480519,32756.432697,38350.610267
358,15.46073,331.093602,5534.941249,33087.526299,38697.164599


In [8]:
annual_growth_rates: list = list()
principal_values: list = list()
monthly_deposits: list = list()

for row_index, row in holdings_profits.iterrows():
    company_name: str = row['name']
    symbol_valuation: ValuationResult = asset_valuation_engine.valuate(company_name=company_name, symbol=row['symbol'])
    annual_growth_rate: float = symbol_valuation.eps_five_years

    if symbol_valuation.annual_dividend_percentage is not None:
        annual_growth_rate += (symbol_valuation.annual_dividend_percentage / 100)

    annual_growth_rates.append(annual_growth_rate)
    principal_values.append(row['total_current_usd'])
    monthly_deposits.append((1000 / usd_zar_exchange_rate / len(holdings_profits)))

holdings_growth: pd.DataFrame = instance.project_monthly_holdings_growth(n_months=n_months,
                                                                         annual_growth_rates=annual_growth_rates,
                                                                         principal_values=principal_values,
                                                                         monthly_deposits=monthly_deposits)

holdings_growth

INFO:root:Fetching HTML page from URL "https://finviz.com/quote.ashx?t=TSLA".
DEBUG:root:User agent override found in config. Using value "PostmanRuntime/7.29.0".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): finviz.com:443
DEBUG:urllib3.connectionpool:https://finviz.com:443 "GET /quote.ashx?t=TSLA HTTP/1.1" 200 None
DEBUG:root:EPS: 8.33, EPS Next 5 Years: 0.5189%
DEBUG:root:P/E Ratio: 110.39, Current Price: $ 919.69
DEBUG:root:Intrinsic Value: $ 707.3451538461537 vs. Current Price: $ 919.69
INFO:root:Fetching HTML page from URL "https://finviz.com/quote.ashx?t=AAPL".
DEBUG:root:User agent override found in config. Using value "PostmanRuntime/7.29.0".
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): finviz.com:443
DEBUG:urllib3.connectionpool:https://finviz.com:443 "GET /quote.ashx?t=AAPL HTTP/1.1" 200 None
DEBUG:root:EPS: 6.05, EPS Next 5 Years: 0.0983%
DEBUG:root:P/E Ratio: 28.59, Current Price: $ 173.03
DEBUG:root:Intrinsic Value: $ 133.0534615384615

,month,deposits_withdrawals,interest,total_deposits_withdrawals,accrued_interest,balance
0,1,61.842919,0.023660,61.842919,6.258309e+00,3.263535e+02
1,2,61.842919,0.023503,123.685838,1.411306e+01,3.960512e+02
2,3,61.842919,0.023427,185.528757,2.361381e+01,4.673949e+02
3,4,61.842919,0.023398,247.371676,3.481197e+01,5.404359e+02
4,5,61.842919,0.023399,309.214595,4.776078e+01,6.152277e+02
...,...,...,...,...,...,...
355,356,61.842919,0.041391,22016.079159,1.481078e+09,1.481101e+09
356,357,61.842919,0.041392,22077.922078,1.545031e+09,1.545053e+09
357,358,61.842919,0.041393,22139.764997,1.611746e+09,1.611769e+09
358,359,61.842919,0.041394,22201.607916,1.681345e+09,1.681367e+09


In [9]:
over_1b = holdings_growth.loc[holdings_growth['balance'] > 1000000000]

print(f'{round(over_1b.index[0] / 12, 2)} years to ${over_1b.iloc[0]["balance"]:.2f}.')

28.83 years to $1012459917.09.


In [10]:
data = {
    'Target Balance (USD)': list(),
    'ETA (Months)': list(),
    'ETA (Years)': list()
}

# $ 100 000
hundred_thousand: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000]

data['Target Balance (USD)'].append(f'{hundred_thousand.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(hundred_thousand.index[0])
data['ETA (Years)'].append(hundred_thousand.index[0] / 12)

# 100 000 0
million: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10]

data['Target Balance (USD)'].append(f'{million.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(million.index[0])
data['ETA (Years)'].append(million.index[0] / 12)

# 100 000 00
ten_million: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10*10]

data['Target Balance (USD)'].append(f'{ten_million.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(ten_million.index[0])
data['ETA (Years)'].append(ten_million.index[0] / 12)

# 100 000 00
hundred_million: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10*100]

data['Target Balance (USD)'].append(f'{hundred_million.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(hundred_million.index[0])
data['ETA (Years)'].append(hundred_million.index[0] / 12)

# 100 000 000
billion: int = holdings_growth.loc[holdings_growth['balance'] > 100*1000*10*1000]

data['Target Balance (USD)'].append(f'{billion.iloc[0]["balance"]:.2f}')
data['ETA (Months)'].append(billion.index[0])
data['ETA (Years)'].append(billion.index[0] / 12)

pd.DataFrame(data)

,Target Balance (USD),ETA (Months),ETA (Years)
0,103407.59,123,10.250000
1,1017435.23,181,15.083333
2,10292811.74,237,19.750000
3,103626985.31,292,24.333333
4,1012459917.09,346,28.833333
